# Key-Recovery Plaintext-Checking Attack on Kyber
$m[j] = \texttt{Compress}(1, v[j] - s_i[j] \cdot u_i[j])$

Depending on the value of $s_i[j]$ (the $j^\text{th}$ coefficient of the $i^\text{th}$ polynomial in the secret key $\mathbf{s}\in R^k$), the range of $v[j]$ for which $m[j]$ is $1$ will change, so depending on the response for some chosen values of $v[j]$, we can pin down the value of $s_i[j]$. We then only need to repeat for all $1 \leq i \leq k, 1 \leq j \leq n$.

## Compression and decompression
```rust
use std::collections::{HashMap, HashSet};
pub const KYBER_Q: u32 = 3329;
pub const HALF_Q: u32 = 1665;

pub fn to_positive_repr(mut val: i16) -> u32 {
    val += (val >> 15) & 3329;
    val as u32
}

pub fn compress(d: usize, mut val: u32) -> u32 {
    val = val << d;
    val += HALF_Q;
    val *= 80635;
    val >>= 28;
    return val & (u32::MAX >> (32 - d));
}

pub fn decompress(d: usize, mut val: u32) -> u32 {
    val = val * KYBER_Q;
    val += 1 << (d - 1);
    val >>= d;
    return val;
}

fn main() {
    // ML-KEM-512/768
    let (du, dv, eta1) = (10usize, 4usize, 3i16);
    // ML-KEM-1024
    // let (du, dv, eta1) = (11usize, 5usize, 2i16);
    let compressed_u = 1<<5;
    let u = decompress(du, compressed_u);
    
    for s in -eta1..=eta1 {
        println!("s: {s}");
        let s_unsigned = to_positive_repr(s);
        for compressed_v in 0..(1<<dv) {
            let v = decompress(dv, compressed_v);
            let decryption = compress(
                1, 
                (v + KYBER_Q - (s_unsigned * u % KYBER_Q)) % KYBER_Q
            );
            println!("compressed_v: {compressed_v}, m: {decryption}");
        }
    }
}
```